<a href="https://colab.research.google.com/github/lis-r-barreto/Data-Engineering/blob/main/01_Profitable_App_Profiles_for_the_App_Store_and_Google_Play_Markets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Perfis de Aplicativo Lucrativos para os Mercados da App Store e Google Play**

*(Profitable App Profiles for the App Store and Google Play Markets)*


<p align="center">
  <img src="https://logowik.com/content/uploads/images/2042-app-store-google-play1.jpg" width="300" >
</p>

## **Objetivo**

Nosso objetivo neste projeto é encontrar perfis de aplicativos móveis que sejam lucrativos para os mercados da App Store e Google Play. Estamos trabalhando como analistas de dados para uma empresa que desenvolve aplicativos para dispositivos móveis Android e iOS, e nosso trabalho é permitir que nossa equipe de desenvolvedores tome decisões baseadas em dados com relação ao tipo de aplicativo que constroem.

Em nossa empresa, só criamos aplicativos de download e instalação gratuitos, e nossa principal fonte de receita consiste em anúncios no aplicativo. Isso significa que nossa receita para qualquer aplicativo é influenciada principalmente pelo número de usuários que usam nosso aplicativo. Nosso objetivo para este projeto é analisar dados para ajudar nossos desenvolvedores a entender quais tipos de aplicativos podem atrair mais usuários.

## **Explorando os Dados**

Em setembro de 2018, havia aproximadamente 2 milhões de aplicativos iOS disponíveis na App Store e 2,1 milhões de aplicativos Android no Google Play.

<p align="center">
  <img src="https://s3.amazonaws.com/dq-content/350/py1m8_statista.png">
</p>

Fonte: [Statista](https://www.statista.com/statistics/276623/number-of-apps-available-in-leading-app-stores/)

Coletar dados para mais de quatro milhões de aplicativos requer uma quantidade significativa de tempo e dinheiro, então, tentaremos analisar uma amostra de dados. Para evitar o gasto de recursos com a coleta de novos dados por conta própria, devemos primeiro tentar ver se podemos encontrar quaisquer dados existentes relevantes sem nenhum custo. Felizmente, esses são dois conjuntos de dados que parecem adequados para o nosso propósito:

- [Um conjunto de dados](https://www.kaggle.com/lava18/google-play-store-apps) contendo dados sobre aproximadamente dez mil aplicativos Android do Google Play. Você pode baixar o conjunto de dados através [desse link](https://dq-content.s3.amazonaws.com/350/googleplaystore.csv).
- [Um conjunto de dados](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps) contendo dados sobre aproximadamente sete mil aplicativos iOS da App Store. Você pode baixar o conjunto de dados através [desse link](https://dq-content.s3.amazonaws.com/350/AppleStore.csv).

Vamos começar abrindo os dois conjuntos de dados e, em seguida, continuar explorando os dados.

In [1]:
# Download do http para arquivo local
!wget --quiet --show-progress https://dq-content.s3.amazonaws.com/350/googleplaystore.csv
!wget --quiet --show-progress https://dq-content.s3.amazonaws.com/350/AppleStore.csv

googleplaystore.csv 100%[===================>]   1.29M  --.-KB/s    in 0.05s   
AppleStore.csv.2    100%[===================>] 708.75K  --.-KB/s    in 0.04s   


In [2]:
from csv import reader

### Dataset do Google Play
opened_file = open('googleplaystore.csv')
read_file = reader(opened_file)
android = list(read_file)
android_header = android[0]
android = android[1:]

### Dataset da App Store
opened_file = open('AppleStore.csv')
read_file = reader(opened_file)
ios = list(read_file)
ios_header = ios[0]
ios = ios[1:]

Para tornar mais fácil explorar os dois conjuntos de dados, primeiro escreveremos uma função chamada `explore_data ()` que podemos usar repetidamente para explorar as linhas de uma forma mais legível. Também adicionaremos uma opção para nossa função para mostrar o número de linhas e colunas para qualquer conjunto de dados.

In [3]:
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n') # Adiciona uma nova linha vazia entre as outras linhas
        
    if rows_and_columns:
        print('Número de linhas:', len(dataset))
        print('Número de colunas:', len(dataset[0]))

print(android_header)
print('\n')
explore_data(android, 0, 3, True)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


Número de linhas: 10841
Número de colunas: 13


Vemos que o conjunto de dados do Google Play tem 10841 aplicativos e 13 colunas. Em uma rápida olhada, as colunas que podem ser úteis para o propósito de nossa análise são `'App'`, `'Category'`, `'Reviews'`, `'Installs'`, `'Type'`, `'Price'`, e `'Genres'`.

Agora vamos dar uma olhada no conjunto de dados da App Store.

In [4]:
print(ios_header)
print('\n')
explore_data(ios, 0, 3, True)

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


Número de linhas: 7197
Número de colunas: 16


Temos 7197 aplicativos iOS neste conjunto de dados e as colunas que parecem interessantes são: 'track_name', 'currency', 'price', 'rating_count_tot', 'rating_count_ver' e 'prime_genre'. Nem todos os nomes de coluna são autoexplicativos neste caso, mas os detalhes sobre cada coluna podem ser encontrados na [documentação do conjunto de dados](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps/home).

## **Deletando Dados Errados**

O conjunto de dados do Google Play tem uma [seção de discussão](https://www.kaggle.com/lava18/google-play-store-apps/discussion) dedicada e podemos ver que [uma das discussões](https://www.kaggle.com/lava18/google-play-store-apps/discussion/66015) descreve um erro para a linha 10472. Vamos imprimir esta linha e compará-la com o cabeçalho e outra linha que está correta.

In [5]:
print(android[10472])  # linha incorreta
print('\n')
print(android_header)  # cabeçalho
print('\n')
print(android[0])      # linha correta

['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']


['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


A linha 10472 corresponde ao aplicativo _Life Made WI-Fi Touchscreen Photo Frame_, e podemos ver que a classificação é 19. Isso está claramente desativado porque a classificação máxima para um aplicativo do Google Play é 5 (conforme mencionado na [seção de discussões](https://www.kaggle.com/lava18/google-play-store-apps/discussion/66015), este problema é causado por um valor ausente na coluna `'Categoria'`). Como consequência, vamos deletar esta linha.

In [6]:
print(len(android))
del android[10472]  # não execute essa linha de código mais de uma vez
print(len(android))

10841
10840


## **Removendo Entradas Duplicadas**

### **Parte I**
Se explorarmos o conjunto de dados do Google Play por tempo suficiente, descobriremos que alguns aplicativos têm mais de uma entrada. Por exemplo, o aplicativo Instagram possui quatro entradas:

In [7]:
for app in android:
    name = app[0]
    if name == 'Instagram':
        print(app)

['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577446', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66509917', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']


No total, existem 1.181 casos em que um aplicativo ocorre mais de uma vez:

In [8]:
duplicate_apps = []
unique_apps = []

for app in android:
    name = app[0]
    if name in unique_apps:
        duplicate_apps.append(name)
    else:
        unique_apps.append(name)
    
print('Número de aplicativos duplicados:', len(duplicate_apps))
print('\n')
print('Examplos de aplicativos duplicados:', duplicate_apps[:15])

Número de aplicativos duplicados: 1181


Examplos de aplicativos duplicados: ['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings', 'Box', 'Zenefits', 'Google Ads', 'Google My Business', 'Slack', 'FreshBooks Classic', 'Insightly CRM', 'QuickBooks Accounting: Invoicing & Expenses', 'HipChat - Chat Built for Teams', 'Xero Accounting Software']


Não queremos contar determinados aplicativos mais de uma vez quando analisamos os dados, então precisamos remover as entradas duplicadas e manter apenas uma entrada por aplicativo. Uma coisa que poderíamos fazer é remover as linhas duplicadas aleatoriamente, mas provavelmente poderíamos encontrar uma maneira melhor.

Se você examinar as linhas que imprimimos nas duas células acima para o aplicativo Instagram, a principal diferença acontece na quarta posição de cada linha, que corresponde ao número de comentários. Os diferentes números mostram que os dados foram coletados em momentos diferentes. Podemos usar isso para construir um critério para manter as linhas. Não removeremos as linhas aleatoriamente, mas manteremos as linhas com o maior número de revisões, pois quanto maior o número de revisões, mais confiáveis ​​serão as avaliações.

Para fazer isso, vamos:

- Criar um dicionário em que cada chave seja um nome de aplicativo exclusivo e o valor seja o maior número de avaliações desse aplicativo;
- Usar o dicionário para criar um novo conjunto de dados, que terá apenas uma entrada por aplicativo (e apenas selecionamos os aplicativos com o maior número de avaliações)

### **Parte II**
Vamos começar construindo o dicionário.

In [9]:
reviews_max = {}

for app in android:
    name = app[0]
    n_reviews = float(app[3])
    
    if name in reviews_max and reviews_max[name] < n_reviews:
        reviews_max[name] = n_reviews
        
    elif name not in reviews_max:
        reviews_max[name] = n_reviews

Em uma célula de código anterior, descobrimos que existem 1.181 casos em que um aplicativo ocorre mais de uma vez, portanto, o comprimento do nosso dicionário (de aplicativos exclusivos) deve ser igual à diferença entre o comprimento do nosso conjunto de dados e 1.181.

In [10]:
print('Tamanho esperado:', len(android) - 1181)
print('Tamanho atual:', len(reviews_max))

Tamanho esperado: 9659
Tamanho atual: 9659


Agora, vamos usar o dicionário `reviews_max` para remover as duplicatas. Para os casos duplicados, manteremos apenas as entradas com o maior número de revisões. Na célula de código abaixo:

- Começamos inicializando duas listas vazias, `android_clean` e` already_added`.
- Fazemos loop através do conjunto de dados `android` e para cada iteração:
    - Isolamos o nome do aplicativo e o número de avaliações.
    - Adicionamos a linha atual (`app`) à lista` android_clean`, e o nome do aplicativo (`name`) à lista` already_added` se:
        - O número de comentários do aplicativo atual corresponde ao número de comentários desse aplicativo, conforme descrito no dicionário `reviews_max`; e
        - O nome do aplicativo ainda não está na lista `already_added`. Precisamos adicionar essa condição suplementar para contabilizar os casos em que o maior número de avaliações de um aplicativo duplicado é o mesmo para mais de uma entrada (por exemplo, o aplicativo Box tem três entradas e o número de avaliações é o mesmo) . Se apenas verificarmos por `reviews_max [name] == n_reviews`, ainda acabaremos com entradas duplicadas para alguns aplicativos.

In [11]:
android_clean = []
already_added = []

for app in android:
    name = app[0]
    n_reviews = float(app[3])
    
    if (reviews_max[name] == n_reviews) and (name not in already_added):
        android_clean.append(app)
        already_added.append(name) # certifique-se de estar dentro do bloco

Agora vamos explorar rapidamente o novo conjunto de dados e confirmar se o número de linhas é 9.659.

In [12]:
explore_data(android_clean, 0, 3, True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


Número de linhas: 9659
Número de colunas: 13


Temos 9659 linhas, exatamente como o esperado.


## **Removendo aplicativos que não sejam em inglês**

### **Parte I**

Se você explorar os conjuntos de dados o suficiente, notará que os nomes de alguns dos aplicativos sugerem que eles não são direcionados a um público que fala inglês. Abaixo, vemos alguns exemplos de ambos os conjuntos de dados:

In [13]:
print(ios[813][1])
print(ios[6731][1])

print(android_clean[4412][0])
print(android_clean[7940][0])

爱奇艺PPS -《欢乐颂2》电视剧热播
【脱出ゲーム】絶対に最後までプレイしないで 〜謎解き＆ブロックパズル〜
中国語 AQリスニング
لعبة تقدر تربح DZ


Não estamos interessados em manter esse tipo de aplicativo, então vamos removê-lo. Uma maneira de fazer isso é remover cada aplicativo cujo nome contém um símbolo que não é comumente usado no texto em inglês - o texto em inglês geralmente inclui letras do alfabeto inglês, números compostos por dígitos de 0 a 9, sinais de pontuação (.,! ,?,;, etc.) e outros símbolos (+, *, /, etc.).

Todos esses caracteres que são específicos para textos em inglês são codificados usando o padrão ASCII. Cada caractere ASCII tem um número correspondente entre 0 e 127 associado a ele, e podemos aproveitar isso para construir uma função que verifica o nome de um aplicativo e nos diz se ele contém caracteres não ASCII.

Construímos esta função abaixo e usamos a função incorporada `ord ()` para descobrir o número de codificação correspondente de cada caractere.

In [14]:
def is_english(string):
    
    for character in string:
        if ord(character) > 127:
            return False
    
    return True

print(is_english('Instagram'))
print(is_english('爱奇艺PPS -《欢乐颂2》电视剧热播'))

True
False


A função parece funcionar bem, mas alguns nomes de aplicativos em inglês usam emojis ou outros símbolos (™, - (traço), - (traço), etc.) que estão fora da faixa ASCII. Por causa disso, removeremos aplicativos úteis se usarmos a função em sua forma atual.

In [15]:
print(is_english('Docs To Go™ Free Office Suite'))
print(is_english('Instachat 😜'))

print(ord('™'))
print(ord('😜'))

False
False
8482
128540


### **Parte II**

Para minimizar o impacto da perda de dados, removeremos um aplicativo apenas se seu nome tiver mais de três caracteres não ASCII:

In [16]:
def is_english(string):
    non_ascii = 0
    
    for character in string:
        if ord(character) > 127:
            non_ascii += 1
    
    if non_ascii > 3:
        return False
    else:
        return True

print(is_english('Docs To Go™ Free Office Suite'))
print(is_english('Instachat 😜'))

True
True


A função ainda não é perfeita e muito poucos aplicativos que não sejam em inglês podem passar por nosso filtro, mas isso parece bom o suficiente neste ponto de nossa análise - não devemos gastar muito tempo com otimização neste ponto.

Abaixo, usamos a função `is_english ()` para filtrar os aplicativos em outros idiomas para ambos os conjuntos de dados:

In [17]:
android_english = []
ios_english = []

for app in android_clean:
    name = app[0]
    if is_english(name):
        android_english.append(app)
        
for app in ios:
    name = app[1]
    if is_english(name):
        ios_english.append(app)
        
explore_data(android_english, 0, 3, True)
print('\n')
explore_data(ios_english, 0, 3, True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


Número de linhas: 9614
Número de colunas: 13


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+'

Podemos ver que ficamos com 9.614 aplicativos para Android e 6.183 aplicativos para iOS.

## **Isolando os aplicativos gratuitos**

Como mencionamos na introdução, apenas criamos aplicativos cujo download e instalação são gratuitos, e nossa principal fonte de receita consiste em anúncios no aplicativo. Nossos conjuntos de dados contêm aplicativos gratuitos e não livres, e precisaremos isolar apenas os aplicativos gratuitos para nossa análise. Abaixo, isolamos os aplicativos gratuitos para ambos os nossos conjuntos de dados.

In [18]:
android_final = []
ios_final = []

for app in android_english:
    price = app[7]
    if price == '0':
        android_final.append(app)
        
for app in ios_english:
    price = app[4]
    if price == '0.0':
        ios_final.append(app)
        
print(len(android_final))
print(len(ios_final))

8864
3222


Ficamos com 8864 aplicativos para Android e 3222 aplicativos para iOS, o que deve ser suficiente para nossa análise.


## **Aplicativos mais comuns por gênero**

### **Parte I**

Como mencionamos na introdução, nosso objetivo é determinar os tipos de aplicativos que provavelmente atrairão mais usuários, pois nossa receita é altamente influenciada pelo número de pessoas que usam nossos aplicativos.

Para minimizar riscos e sobrecarga, nossa estratégia de validação para uma ideia de aplicativo é composta de três etapas:

1. Crie uma versão Android mínima do aplicativo e adicione-a ao Google Play.
2. Se o aplicativo tiver uma boa resposta dos usuários, nós o desenvolveremos ainda mais.
3. Se o aplicativo for lucrativo após seis meses, também criamos uma versão iOS do aplicativo e a adicionamos à App Store.

Como nosso objetivo final é adicionar o aplicativo na App Store e no Google Play, precisamos encontrar perfis de aplicativos que tenham sucesso em ambos os mercados. Por exemplo, um perfil que pode funcionar bem para os dois mercados pode ser um aplicativo de produtividade que faz uso de gamificação.

Vamos começar a análise obtendo uma noção dos gêneros mais comuns para cada mercado. Para isso, construiremos uma tabela de frequência para a coluna `prime_genre` do conjunto de dados da App Store e as colunas` Genres` e `Category` do conjunto de dados do Google Play.


### **Parte II**

Vamos construir duas funções que podemos usar para analisar as tabelas de frequência:

- Uma função para gerar tabelas de frequência que mostram percentagens
- Outra função que podemos usar para exibir as porcentagens em ordem decrescente

In [19]:
def freq_table(dataset, index):
    table = {}
    total = 0
    
    for row in dataset:
        total += 1
        value = row[index]
        if value in table:
            table[value] += 1
        else:
            table[value] = 1
    
    table_percentages = {}
    for key in table:
        percentage = (table[key] / total) * 100
        table_percentages[key] = percentage 
    
    return table_percentages


def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)
        
    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])

### **Parte III**

Começamos examinando a tabela de frequência para a coluna `prime_genre` do conjunto de dados da App Store.

In [20]:
display_table(ios_final, -5)

Games : 58.16263190564867
Entertainment : 7.883302296710118
Photo & Video : 4.9658597144630665
Education : 3.662321539416512
Social Networking : 3.2898820608317814
Shopping : 2.60707635009311
Utilities : 2.5139664804469275
Sports : 2.1415270018621975
Music : 2.0484171322160147
Health & Fitness : 2.0173805090006205
Productivity : 1.7380509000620732
Lifestyle : 1.5828677839851024
News : 1.3345747982619491
Travel : 1.2414649286157666
Finance : 1.1173184357541899
Weather : 0.8690254500310366
Food & Drink : 0.8069522036002483
Reference : 0.5586592178770949
Business : 0.5276225946617008
Book : 0.4345127250155183
Navigation : 0.186219739292365
Medical : 0.186219739292365
Catalogs : 0.12414649286157665


Podemos verificar que entre os aplicativos gratuitos em inglês, mais da metade (58,16%) são jogos. Os aplicativos de entretenimento estão perto de 8%, seguidos pelos aplicativos de foto e vídeo, que estão perto de 5%. Apenas 3,66% dos aplicativos são projetados para a educação, seguidos por aplicativos de redes sociais que somam 3,29% dos aplicativos em nosso conjunto de dados.

A impressão geral é que a App Store (pelo menos a parte que contém aplicativos em inglês gratuitos) é dominada por aplicativos que são projetados para se divertir (jogos, entretenimento, foto e vídeo, redes sociais, esportes, música, etc.), enquanto aplicativos com práticas propósitos (educação, compras, serviços públicos, produtividade, estilo de vida, etc.) são mais raros. No entanto, o fato de os aplicativos divertidos serem os mais numerosos também não significa que eles também tenham o maior número de usuários - a demanda pode não ser a mesma que a oferta.

Vamos continuar examinando as colunas `Genres` e` Category` do conjunto de dados do Google Play (duas colunas que parecem estar relacionadas).

In [21]:
display_table(android_final, 1) # Category

FAMILY : 18.907942238267147
GAME : 9.724729241877256
TOOLS : 8.461191335740072
BUSINESS : 4.591606498194946
LIFESTYLE : 3.9034296028880866
PRODUCTIVITY : 3.892148014440433
FINANCE : 3.7003610108303246
MEDICAL : 3.531137184115524
SPORTS : 3.395758122743682
PERSONALIZATION : 3.3167870036101084
COMMUNICATION : 3.2378158844765346
HEALTH_AND_FITNESS : 3.0798736462093865
PHOTOGRAPHY : 2.944494584837545
NEWS_AND_MAGAZINES : 2.7978339350180503
SOCIAL : 2.6624548736462095
TRAVEL_AND_LOCAL : 2.33528880866426
SHOPPING : 2.2450361010830324
BOOKS_AND_REFERENCE : 2.1435018050541514
DATING : 1.861462093862816
VIDEO_PLAYERS : 1.7937725631768955
MAPS_AND_NAVIGATION : 1.3989169675090252
FOOD_AND_DRINK : 1.2409747292418771
EDUCATION : 1.1620036101083033
ENTERTAINMENT : 0.9589350180505415
LIBRARIES_AND_DEMO : 0.9363718411552346
AUTO_AND_VEHICLES : 0.9250902527075812
HOUSE_AND_HOME : 0.8235559566787004
WEATHER : 0.8009927797833934
EVENTS : 0.7107400722021661
PARENTING : 0.6543321299638989
ART_AND_DESIGN : 

O cenário parece significativamente diferente no Google Play: não existem muitos aplicativos projetados para se divertir, e parece que um bom número de aplicativos são projetados para fins práticos (família, ferramentas, negócios, estilo de vida, produtividade, etc.). No entanto, se investigarmos isso mais a fundo, podemos ver que a categoria família (que responde por quase 19% dos aplicativos) significa principalmente jogos para crianças.


<p align="center">
  <img src="https://s3.amazonaws.com/dq-content/350/py1m8_family.png">
</p>

Mesmo assim, os aplicativos práticos parecem ter uma melhor representação no Google Play em comparação com a App Store. Esta imagem também é confirmada pela tabela de frequência que vemos na coluna `Genres`:

In [22]:
display_table(android_final, -4)

Tools : 8.449909747292418
Entertainment : 6.069494584837545
Education : 5.347472924187725
Business : 4.591606498194946
Productivity : 3.892148014440433
Lifestyle : 3.892148014440433
Finance : 3.7003610108303246
Medical : 3.531137184115524
Sports : 3.463447653429603
Personalization : 3.3167870036101084
Communication : 3.2378158844765346
Action : 3.1024368231046933
Health & Fitness : 3.0798736462093865
Photography : 2.944494584837545
News & Magazines : 2.7978339350180503
Social : 2.6624548736462095
Travel & Local : 2.3240072202166067
Shopping : 2.2450361010830324
Books & Reference : 2.1435018050541514
Simulation : 2.0419675090252705
Dating : 1.861462093862816
Arcade : 1.8501805054151623
Video Players & Editors : 1.7712093862815883
Casual : 1.7599277978339352
Maps & Navigation : 1.3989169675090252
Food & Drink : 1.2409747292418771
Puzzle : 1.128158844765343
Racing : 0.9927797833935018
Role Playing : 0.9363718411552346
Libraries & Demo : 0.9363718411552346
Auto & Vehicles : 0.9250902527075

A diferença entre as colunas `Genres` e` Category` não é muito clara, mas uma coisa que podemos notar é que a coluna `Genres` é muito mais granular (tem mais categorias). No momento, estamos apenas procurando o panorama geral, então trabalharemos apenas com a coluna `Category` daqui para frente.

Até este ponto, descobrimos que a App Store é dominada por aplicativos projetados para diversão, enquanto o Google Play mostra um cenário mais equilibrado de aplicativos práticos e divertidos. Agora, gostaríamos de ter uma ideia sobre o tipo de aplicativo que tem a maioria dos usuários.

## **Aplicativos mais populares por gênero na App Store**

Uma maneira de descobrir quais gêneros são os mais populares (têm mais usuários) é calcular o número médio de instalações para cada gênero de aplicativo. Para o conjunto de dados do Google Play, podemos encontrar essas informações na coluna `Installs`, mas para o conjunto de dados da App Store, essas informações estão ausentes. Como solução alternativa, tomaremos o número total de classificações do usuário como um proxy, que podemos encontrar no aplicativo `rating_count_tot`.

Abaixo, calculamos o número médio de classificações de usuários por gênero de aplicativo na App Store:

In [23]:
genres_ios = freq_table(ios_final, -5)

for genre in genres_ios:
    total = 0
    len_genre = 0
    for app in ios_final:
        genre_app = app[-5]
        if genre_app == genre:            
            n_ratings = float(app[5])
            total += n_ratings
            len_genre += 1
    avg_n_ratings = total / len_genre
    print(genre, ':', avg_n_ratings)

Social Networking : 71548.34905660378
Photo & Video : 28441.54375
Games : 22788.6696905016
Music : 57326.530303030304
Reference : 74942.11111111111
Health & Fitness : 23298.015384615384
Weather : 52279.892857142855
Utilities : 18684.456790123455
Travel : 28243.8
Shopping : 26919.690476190477
News : 21248.023255813954
Navigation : 86090.33333333333
Lifestyle : 16485.764705882353
Entertainment : 14029.830708661417
Food & Drink : 33333.92307692308
Sports : 23008.898550724636
Book : 39758.5
Finance : 31467.944444444445
Education : 7003.983050847458
Productivity : 21028.410714285714
Business : 7491.117647058823
Catalogs : 4004.0
Medical : 612.0


Em média, os aplicativos de navegação têm o maior número de avaliações de usuários, mas esse número é fortemente influenciado pelo Waze e pelo Google Maps, que têm quase meio milhão de avaliações de usuários juntas:

In [24]:
for app in ios_final:
    if app[-5] == 'Navigation':
        print(app[1], ':', app[5]) # Imprime nome e número de avaliações

Waze - GPS Navigation, Maps & Real-time Traffic : 345046
Google Maps - Navigation & Transit : 154911
Geocaching® : 12811
CoPilot GPS – Car Navigation & Offline Maps : 3582
ImmobilienScout24: Real Estate Search in Germany : 187
Railway Route Search : 5


O mesmo padrão se aplica a aplicativos de redes sociais, onde o número médio é fortemente influenciado por alguns gigantes como Facebook, Pinterest, Skype, etc. O mesmo se aplica a aplicativos de música, onde alguns grandes jogadores como Pandora, Spotify e Shazam influenciam fortemente o número médio.

Nosso objetivo é encontrar gêneros populares, mas aplicativos de navegação, redes sociais ou música podem parecer mais populares do que realmente são. O número médio de classificações parece ser distorcido por muito poucos aplicativos que têm centenas de milhares de classificações de usuários, enquanto os outros aplicativos podem ter dificuldade em passar do limite de 10.000. Poderíamos ter uma imagem melhor removendo esses aplicativos extremamente populares para cada gênero e, em seguida, refazer as médias, mas deixaremos esse nível de detalhe para mais tarde.

Os aplicativos de referência têm 74.942 avaliações de usuários em média, mas na verdade são a Bíblia e o Dictionary.com que elevam a avaliação média:

In [25]:
for app in ios_final:
    if app[-5] == 'Reference':
        print(app[1], ':', app[5])

Bible : 985920
Dictionary.com Dictionary & Thesaurus : 200047
Dictionary.com Dictionary & Thesaurus for iPad : 54175
Google Translate : 26786
Muslim Pro: Ramadan 2017 Prayer Times, Azan, Quran : 18418
New Furniture Mods - Pocket Wiki & Game Tools for Minecraft PC Edition : 17588
Merriam-Webster Dictionary : 16849
Night Sky : 12122
City Maps for Minecraft PE - The Best Maps for Minecraft Pocket Edition (MCPE) : 8535
LUCKY BLOCK MOD ™ for Minecraft PC Edition - The Best Pocket Wiki & Mods Installer Tools : 4693
GUNS MODS for Minecraft PC Edition - Mods Tools : 1497
Guides for Pokémon GO - Pokemon GO News and Cheats : 826
WWDC : 762
Horror Maps for Minecraft PE - Download The Scariest Maps for Minecraft Pocket Edition (MCPE) Free : 718
VPN Express : 14
Real Bike Traffic Rider Virtual Reality Glasses : 8
教えて!goo : 0
Jishokun-Japanese English Dictionary & Translator : 0


No entanto, esse nicho parece apresentar algum potencial. Uma coisa que poderíamos fazer é pegar outro livro popular e transformá-lo em um aplicativo onde poderíamos adicionar diferentes recursos além da versão bruta do livro. Isso pode incluir citações diárias do livro, uma versão em áudio do livro, questionários sobre o livro, etc. Além disso, também podemos incorporar um dicionário dentro do aplicativo, para que os usuários não precisem sair de nosso aplicativo para olhar palavras em um aplicativo externo.

Essa ideia parece se encaixar bem com o fato de que a App Store é dominada por aplicativos para diversão. Isso sugere que o mercado pode estar um pouco saturado com aplicativos divertidos, o que significa que um aplicativo prático pode ter mais chance de se destacar entre o grande número de aplicativos na App Store.

Outros gêneros que parecem populares incluem clima, livro, comida e bebida ou finanças. O gênero do livro parece se sobrepor um pouco à ideia do aplicativo que descrevemos acima, mas os outros gêneros não nos parecem muito interessantes:

- Aplicativos de clima - as pessoas geralmente não passam muito tempo no aplicativo e as chances de lucrar com adições no aplicativo são baixas. Além disso, obter dados meteorológicos confiáveis ​​ao vivo pode exigir que conectemos nossos aplicativos a APIs não livres.

- Alimentos e bebidas - os exemplos aqui incluem Starbucks, Dunkin 'Donuts, McDonald's, etc. Portanto, fazer um aplicativo popular de comida e bebida requer uma cozinha real e um serviço de entrega, o que está fora do escopo de nossa empresa.

- Aplicativos de finanças - esses aplicativos envolvem serviços bancários, pagamento de contas, transferência de dinheiro, etc. Construir um aplicativo de finanças requer conhecimento de domínio e não queremos contratar um especialista em finanças apenas para construir um aplicativo.

Agora vamos analisar um pouco o mercado do Google Play.

## **Aplicativos mais populares por gênero no Google Play**

Para o mercado do Google Play, temos dados sobre o número de instalações, então devemos ser capazes de ter uma ideia mais clara sobre a popularidade do gênero. No entanto, os números de instalação não parecem precisos o suficiente - podemos ver que a maioria dos valores é aberta (100+, 1.000+, 5.000+, etc.):

In [26]:
display_table(android_final, 5) # A coluna 'Installs'

1,000,000+ : 15.726534296028879
100,000+ : 11.552346570397113
10,000,000+ : 10.548285198555957
10,000+ : 10.198555956678701
1,000+ : 8.393501805054152
100+ : 6.915613718411552
5,000,000+ : 6.825361010830325
500,000+ : 5.561823104693141
50,000+ : 4.7721119133574
5,000+ : 4.512635379061372
10+ : 3.5424187725631766
500+ : 3.2490974729241873
50,000,000+ : 2.3014440433213
100,000,000+ : 2.1322202166064983
50+ : 1.917870036101083
5+ : 0.78971119133574
1+ : 0.5076714801444043
500,000,000+ : 0.2707581227436823
1,000,000,000+ : 0.22563176895306858
0+ : 0.04512635379061372
0 : 0.01128158844765343


Um problema com esses dados é que não são precisos. Por exemplo, não sabemos se um aplicativo com mais de 100.000 instalações possui 100.000 instalações, 200.000 ou 350.000. No entanto, não precisamos de dados muito precisos para nossos propósitos - queremos apenas ter uma ideia de quais gêneros de aplicativos atraem mais usuários e não precisamos de precisão perfeita com relação ao número de usuários.

Vamos deixar os números como estão, o que significa que consideraremos que um aplicativo com mais de 100.000 instalações tem 100.000 instalações e um aplicativo com mais de 1.000.000 de instalações tem 1.000.000 de instalações e assim por diante.

Para realizar cálculos, no entanto, precisaremos converter cada número de instalação em `float` - isso significa que precisamos remover as vírgulas e os caracteres de adição, caso contrário, a conversão falhará e gerará um erro. Faremos isso diretamente no loop abaixo, onde também calculamos o número médio de instalações para cada gênero (categoria).

In [27]:
categories_android = freq_table(android_final, 1)

for category in categories_android:
    total = 0
    len_category = 0
    for app in android_final:
        category_app = app[1]
        if category_app == category:            
            n_installs = app[5]
            n_installs = n_installs.replace(',', '')
            n_installs = n_installs.replace('+', '')
            total += float(n_installs)
            len_category += 1
    avg_n_installs = total / len_category
    print(category, ':', avg_n_installs)

ART_AND_DESIGN : 1986335.0877192982
AUTO_AND_VEHICLES : 647317.8170731707
BEAUTY : 513151.88679245283
BOOKS_AND_REFERENCE : 8767811.894736841
BUSINESS : 1712290.1474201474
COMICS : 817657.2727272727
COMMUNICATION : 38456119.167247385
DATING : 854028.8303030303
EDUCATION : 1833495.145631068
ENTERTAINMENT : 11640705.88235294
EVENTS : 253542.22222222222
FINANCE : 1387692.475609756
FOOD_AND_DRINK : 1924897.7363636363
HEALTH_AND_FITNESS : 4188821.9853479853
HOUSE_AND_HOME : 1331540.5616438356
LIBRARIES_AND_DEMO : 638503.734939759
LIFESTYLE : 1437816.2687861272
GAME : 15588015.603248259
FAMILY : 3695641.8198090694
MEDICAL : 120550.61980830671
SOCIAL : 23253652.127118643
SHOPPING : 7036877.311557789
PHOTOGRAPHY : 17840110.40229885
SPORTS : 3638640.1428571427
TRAVEL_AND_LOCAL : 13984077.710144928
TOOLS : 10801391.298666667
PERSONALIZATION : 5201482.6122448975
PRODUCTIVITY : 16787331.344927534
PARENTING : 542603.6206896552
WEATHER : 5074486.197183099
VIDEO_PLAYERS : 24727872.452830188
NEWS_AND_

Em média, os aplicativos de comunicação têm o maior número de instalações: 38.456.119. Este número é fortemente distorcido por alguns aplicativos que têm mais de um bilhão de instalações (WhatsApp, Facebook Messenger, Skype, Google Chrome, Gmail e Hangouts) e alguns outros com mais de 100 e 500 milhões de instalações:

In [28]:
for app in android_final:
    if app[1] == 'COMMUNICATION' and (app[5] == '1,000,000,000+'
                                      or app[5] == '500,000,000+'
                                      or app[5] == '100,000,000+'):
        print(app[0], ':', app[5])

WhatsApp Messenger : 1,000,000,000+
imo beta free calls and text : 100,000,000+
Android Messages : 100,000,000+
Google Duo - High Quality Video Calls : 500,000,000+
Messenger – Text and Video Chat for Free : 1,000,000,000+
imo free video calls and chat : 500,000,000+
Skype - free IM & video calls : 1,000,000,000+
Who : 100,000,000+
GO SMS Pro - Messenger, Free Themes, Emoji : 100,000,000+
LINE: Free Calls & Messages : 500,000,000+
Google Chrome: Fast & Secure : 1,000,000,000+
Firefox Browser fast & private : 100,000,000+
UC Browser - Fast Download Private & Secure : 500,000,000+
Gmail : 1,000,000,000+
Hangouts : 1,000,000,000+
Messenger Lite: Free Calls & Messages : 100,000,000+
Kik : 100,000,000+
KakaoTalk: Free Calls & Text : 100,000,000+
Opera Mini - fast web browser : 100,000,000+
Opera Browser: Fast and Secure : 100,000,000+
Telegram : 100,000,000+
Truecaller: Caller ID, SMS spam blocking & Dialer : 100,000,000+
UC Browser Mini -Tiny Fast Private & Secure : 100,000,000+
Viber Mess

Se removêssemos todos os aplicativos de comunicação com mais de 100 milhões de instalações, a média seria reduzida cerca de dez vezes:

In [29]:
under_100_m = []

for app in android_final:
    n_installs = app[5]
    n_installs = n_installs.replace(',', '')
    n_installs = n_installs.replace('+', '')
    if (app[1] == 'COMMUNICATION') and (float(n_installs) < 100000000):
        under_100_m.append(float(n_installs))
        
sum(under_100_m) / len(under_100_m)

3603485.3884615386

Vemos o mesmo padrão para a categoria de players de vídeo, que é a segunda colocada com 24.727.872 instalações. O mercado é dominado por aplicativos como Youtube, Google Play Filmes e TV ou MX Player. O padrão é repetido para aplicativos sociais (onde temos gigantes como Facebook, Instagram, Google+, etc.), aplicativos de fotografia (Google Fotos e outros editores de fotos populares) ou aplicativos de produtividade (Microsoft Word, Dropbox, Google Calendar, Evernote, etc. .).

Novamente, a principal preocupação é que esses gêneros de aplicativos possam parecer mais populares do que realmente são. Além disso, esses nichos parecem ser dominados por alguns gigantes contra os quais é difícil competir.

O gênero de jogo parece bastante popular, mas antes descobrimos que essa parte do mercado parece um pouco saturada, então gostaríamos de sugerir um aplicativo diferente, se possível.

Os livros e o gênero de referência também parecem bastante populares, com um número médio de instalações de 8.767.811. É interessante explorar isso com mais profundidade, pois descobrimos que esse gênero tem algum potencial para funcionar bem na App Store, e nosso objetivo é recomendar um gênero de aplicativo que mostra potencial para ser lucrativo tanto na App Store quanto no Google Play.

Vamos dar uma olhada em alguns dos aplicativos desse gênero e seu número de instalações:

In [30]:
for app in android_final:
    if app[1] == 'BOOKS_AND_REFERENCE':
        print(app[0], ':', app[5])

E-Book Read - Read Book for free : 50,000+
Download free book with green book : 100,000+
Wikipedia : 10,000,000+
Cool Reader : 10,000,000+
Free Panda Radio Music : 100,000+
Book store : 1,000,000+
FBReader: Favorite Book Reader : 10,000,000+
English Grammar Complete Handbook : 500,000+
Free Books - Spirit Fanfiction and Stories : 1,000,000+
Google Play Books : 1,000,000,000+
AlReader -any text book reader : 5,000,000+
Offline English Dictionary : 100,000+
Offline: English to Tagalog Dictionary : 500,000+
FamilySearch Tree : 1,000,000+
Cloud of Books : 1,000,000+
Recipes of Prophetic Medicine for free : 500,000+
ReadEra – free ebook reader : 1,000,000+
Anonymous caller detection : 10,000+
Ebook Reader : 5,000,000+
Litnet - E-books : 100,000+
Read books online : 5,000,000+
English to Urdu Dictionary : 500,000+
eBoox: book reader fb2 epub zip : 1,000,000+
English Persian Dictionary : 500,000+
Flybook : 500,000+
All Maths Formulas : 1,000,000+
Ancestry : 5,000,000+
HTC Help : 10,000,000+
E

O livro e o gênero de referência incluem uma variedade de aplicativos: software para processar e ler e-books, várias coleções de bibliotecas, dicionários, tutoriais sobre programação ou linguagens, etc. Parece que ainda há um pequeno número de aplicativos extremamente populares que distorcem a média:

In [31]:
for app in android_final:
    if app[1] == 'BOOKS_AND_REFERENCE' and (app[5] == '1,000,000,000+'
                                            or app[5] == '500,000,000+'
                                            or app[5] == '100,000,000+'):
        print(app[0], ':', app[5])

Google Play Books : 1,000,000,000+
Bible : 100,000,000+
Amazon Kindle : 100,000,000+
Wattpad 📖 Free Books : 100,000,000+
Audiobooks from Audible : 100,000,000+


No entanto, parece que existem apenas alguns aplicativos muito populares, então este mercado ainda mostra potencial. Vamos tentar obter algumas ideias de aplicativos com base no tipo de aplicativo que está em algum lugar no meio em termos de popularidade (entre 1.000.000 e 100.000.000 de downloads):

In [32]:
for app in android_final:
    if app[1] == 'BOOKS_AND_REFERENCE' and (app[5] == '1,000,000+'
                                            or app[5] == '5,000,000+'
                                            or app[5] == '10,000,000+'
                                            or app[5] == '50,000,000+'):
        print(app[0], ':', app[5])

Wikipedia : 10,000,000+
Cool Reader : 10,000,000+
Book store : 1,000,000+
FBReader: Favorite Book Reader : 10,000,000+
Free Books - Spirit Fanfiction and Stories : 1,000,000+
AlReader -any text book reader : 5,000,000+
FamilySearch Tree : 1,000,000+
Cloud of Books : 1,000,000+
ReadEra – free ebook reader : 1,000,000+
Ebook Reader : 5,000,000+
Read books online : 5,000,000+
eBoox: book reader fb2 epub zip : 1,000,000+
All Maths Formulas : 1,000,000+
Ancestry : 5,000,000+
HTC Help : 10,000,000+
Moon+ Reader : 10,000,000+
English-Myanmar Dictionary : 1,000,000+
Golden Dictionary (EN-AR) : 1,000,000+
All Language Translator Free : 1,000,000+
Aldiko Book Reader : 10,000,000+
Dictionary - WordWeb : 5,000,000+
50000 Free eBooks & Free AudioBooks : 5,000,000+
Al-Quran (Free) : 10,000,000+
Al Quran Indonesia : 10,000,000+
Al'Quran Bahasa Indonesia : 10,000,000+
Al Quran Al karim : 1,000,000+
Al Quran : EAlim - Translations & MP3 Offline : 5,000,000+
Koran Read &MP3 30 Juz Offline : 1,000,000+
H

Este nicho parece ser dominado por software para processamento e leitura de e-books, bem como várias coleções de bibliotecas e dicionários, então provavelmente não é uma boa ideia construir aplicativos semelhantes, pois haverá uma concorrência significativa.

Também notamos que existem alguns aplicativos desenvolvidos em torno do livro Alcorão, o que sugere que construir um aplicativo em torno de um livro popular pode ser lucrativo. Parece que pegar um livro popular (talvez um livro mais recente) e transformá-lo em um aplicativo pode ser lucrativo para os mercados do Google Play e da App Store.

No entanto, parece que o mercado já está cheio de bibliotecas, então precisamos adicionar alguns recursos especiais além da versão bruta do livro. Isso pode incluir citações diárias do livro, uma versão em áudio do livro, questionários sobre o livro, um fórum onde as pessoas podem discutir o livro, etc.

## **Conclusões**

Neste projeto, analisamos dados sobre aplicativos móveis da App Store e Google Play com o objetivo de recomendar um perfil de aplicativo que possa ser lucrativo para ambos os mercados.

Concluímos que pegar um livro popular (talvez um livro mais recente) e transformá-lo em um aplicativo pode ser lucrativo para os mercados do Google Play e da App Store. Os mercados já estão cheios de bibliotecas, então precisamos adicionar alguns recursos especiais além da versão bruta do livro. Isso pode incluir citações diárias do livro, uma versão em áudio do livro, questionários sobre o livro, um fórum onde as pessoas possam discutir o livro, etc.


### **Este projeto guiado faz parte do [*Data Enginnering Carreer Path*](https://www.dataquest.io/path/data-engineer/) proposto pela**

<p align="center">
  <img src="https://www.bootcamprating.com/wp-content/uploads/2020/09/Dataquest_school_logo_image.png" width="200" >
</p>